In [173]:
def download_folder(bucket_name, folder_prefix, destination_directory):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_prefix)

    lst = []

    for blob in blobs:
        file_name = blob.name.split('/')[-1]
        local_directory = os.path.join(destination_directory, blob.name[len(folder_prefix):].split('/')[0])
        os.makedirs(local_directory, exist_ok=True)

        lst.append(os.path.join(local_directory, file_name))
        # blob.download_to_filename(file_to_download)
        
        # print(file_to_download)

bucket_name = "dubpro-ai-data"
folder_prefix = "youtube_data/"
destination_directory = "material"
download_folder(bucket_name, folder_prefix, destination_directory)

/home/harsh/miniconda3/envs/env/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/harsh/miniconda3/envs/env/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
import os
import json
import soundfile as sf
from google.cloud import storage

def download_folder(bucket_name, folder_prefix, destination_directory):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_prefix)

    for blob in blobs:
        file_name = blob.name.split('/')[-1]
        local_directory = os.path.join(destination_directory, blob.name[len(folder_prefix):].split('/')[0])
        os.makedirs(local_directory, exist_ok=True)

        file_to_download = os.path.join(local_directory, file_name)
        blob.download_to_filename(file_to_download)

        with open('/home/harsh/Documents/audio_trimming/material/youtube_data_--M8DKL-Vsw_--M8DKL-Vsw.json') as fd:
            info = json.load(fd)

        input_file = "/home/harsh/Documents/audio_trimming/material/youtube_data_--M8DKL-Vsw_--M8DKL-Vsw.flac"
        trim_audio(input_file)

        empty_folder('material')
        empty_folder('output_file')

        
    
# Trim the audio
def trim_audio(input_file):
    # Read the audio file
    data, samplerate = sf.read(input_file)

    # Check if the audio is mono
    if len(data.shape) == 1:

        for idx, item in enumerate(info):
            
            start_time = item["start"]
            end_time = item["end"]
            
            # Calculate start and end samples
            start_sample = int(start_time * samplerate)
            end_sample = int(end_time * samplerate)

            # Trim the audio
            trimmed_data = data[start_sample:end_sample]

            # Write the trimmed audio to a new file

            if not os.path.exists('output_file'):
                os.makedirs('output_file')
            basename = os.path.basename('/home/harsh/Documents/audio_trimming/material/youtube_data_--M8DKL-Vsw_--M8DKL-Vsw.flac')
            path_to_otpt_audio = 'output_file/' + basename[:-5] + '_' + str(idx+1) + '.flac'
            
            sf.write(path_to_otpt_audio, trimmed_data, samplerate)



def empty_folder(folder_path):
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for file in files:
            # Remove each file
            file_path = os.path.join(root, file)
            os.remove(file_path)
        for dir in dirs:
            # Remove each subfolder
            dir_path = os.path.join(root, dir)
            os.rmdir(dir_path)


if __name__ == "__main__":
    bucket_name = "dubpro-ai-data"
    folder_prefix = "youtube_data/"
    destination_directory = "material"
    download_folder(bucket_name, folder_prefix, destination_directory)

## <font color = 'red'>Using AWS SDK - boto3</font>

## Pipeline

In [132]:
import os
import json
import boto3
import soundfile as sf
from dotenv import load_dotenv

load_dotenv(".env")

s3 = boto3.resource("s3", 
                        region_name="auto",
                        endpoint_url='https://storage.googleapis.com',  
                        aws_access_key_id=os.environ.get("HMAC_KEY"),
                        aws_secret_access_key=os.environ.get("HMAC_SECRET"))

BASE_INPUT_DIR = 'youtube_data'
BASE_OUTPUT_DIR = "youtube_trimmed_data"


def download_file(gcs_link, save_path):
    s3_key = gcs_link.replace("gs://", "")
    bucket, key = s3_key.split("/", maxsplit=1)
    s3.meta.client.download_file(
        Bucket=bucket,
        Key=key,
        Filename=save_path,
    )


def download_gcs(gcs_links_and_paths):
    os.makedirs(BASE_INPUT_DIR, exist_ok=True)

    for json_link, audio_link, folder_name in gcs_links_and_paths:
        folder_path = os.path.join(BASE_INPUT_DIR, folder_name)
        os.makedirs(folder_path, exist_ok=True)

        local_json_file = os.path.join(folder_path, os.path.basename(json_link))
        local_audio_file = os.path.join(folder_path, os.path.basename(audio_link))

        print(json_link)

        download_file(json_link, local_json_file)
        download_file(audio_link, local_audio_file)

        save_folder = os.path.join(BASE_OUTPUT_DIR, folder_name)

        trim_audio(local_audio_file, local_json_file, save_folder)



# Trim the audio
def trim_audio(audio_file, json_file, save_folder, extension="flac"):

    # Loading json file
    with open(json_file) as fd:
        info = json.load(fd)

    # Read the audio file
    data, samplerate = sf.read(audio_file)

    # Check if the audio is mono
    if len(data.shape) == 1:

        for idx, item in enumerate(info):
            
            start_time = item["start"]
            end_time = item["end"]
            
            # Calculate start and end samples
            start_sample = int(start_time * samplerate)
            end_sample = int(end_time * samplerate)

            # Trim the audio
            trimmed_data = data[start_sample:end_sample]

            fname = os.path.basename(save_folder)
            
            path_to_otpt_audio = os.path.join(save_folder, f"{fname}_{idx}.{extension}")
            
            sf.write(path_to_otpt_audio, trimmed_data, samplerate)



def empty_folder(folder_path):
    for root, dirs, files in os.walk(folder_path, topdown=False):
        for file in files:
            # Remove each file
            file_path = os.path.join(root, file)
            os.remove(file_path)
        for dir in dirs:
            # Remove each subfolder
            dir_path = os.path.join(root, dir)
            os.rmdir(dir_path)

# Getting the links ready to download flac and json files
gcs_links_and_paths = []
with open('file.txt') as fd:
    d = fd.read().splitlines()
    d = list(filter(lambda x: x.rstrip("/"), d))

for e in d:
    fold_name = e.split('/')[-1]
    json_path = os.path.join(e, f"{fold_name}.json")
    audio_path = os.path.join(e, f"{fold_name}.flac")   
    gcs_links_and_paths.append((json_path, audio_path, fold_name))
    break

download_gcs(gcs_links_and_paths)

gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/.json


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [109]:
with open('file.txt') as fd:
    data = fd.read().splitlines()
data = ' '.join(data)

for e in data.split('\n'):
    input_flac_file = '/home/harsh/Documents/audio_trmming/material/' + e.split('/')[-2] + '.flac'
    input_json_file = '/home/harsh/Documents/audio_trmming/material/' + e.split('/')[-2] + '.json'
    break
input_json_file

'/home/harsh/Documents/audio_trmming/material/zz_H4Wcq3Tg.json'

In [122]:
# Correct

with open('file.txt') as fd:
    data = fd.read().splitlines()
data = ' '.join(data)

for e in data.split():
    input_flac_file = '/home/harsh/Documents/audio_trmming/material/' + e.split('/')[-2] + '.flac'
    input_json_file = '/home/harsh/Documents/audio_trmming/material/' + e.split('/')[-2] + '.json'
    break

input_flac_file,input_json_file

('/home/harsh/Documents/audio_trmming/material/--M8DKL-Vsw.flac',
 '/home/harsh/Documents/audio_trmming/material/--M8DKL-Vsw.json')

In [ ]:
def download_gcs(gcs_links_and_paths):
    load_dotenv(".env")

    s3 = boto3.resource("s3", 
                        region_name="auto",
                        endpoint_url='https://storage.googleapis.com',  
                        aws_access_key_id=os.environ.get("HMAC_KEY"),
                        aws_secret_access_key=os.environ.get("HMAC_SECRET"))

    # for gcs_link, filepath in gcs_links_and_paths:
    #     s3_key = gcs_link.replace("gs://", "")
    #     bucket, key = s3_key.split("/", maxsplit=1)

    if not os.path.exists('material'):
        os.makedirs('material')

    for i in gcs_links_and_paths:
        for gcs_link in i:
            s3_key = gcs_link.replace("gs://", "")
            bucket, key = s3_key.split("/", maxsplit=1)

            s3.meta.client.download_file(
                Filename='material',
                Bucket=bucket,
                Key=key,
            )

# gcs_links_and_paths = [
#     ("gs://dubpro-ai-data/youtube_data/-_hv34i11Zg/-_hv34i11Zg.json", "/home/harsh/Documents/audio_trimming/material/_hv34i11Zg.flac"),
#     ("gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json", "/home/harsh/Documents/audio_trimming/material/--M8DKL-Vsw.flac")
# ]

gcs_links_and_paths = [['gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json',
  'gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.flac']]

download_gcs(gcs_links_and_paths)

In [93]:
# chat
import os
import boto3
from dotenv import load_dotenv

def download_gcs(gcs_links_and_paths):
    load_dotenv(".env")

    s3 = boto3.resource("s3", 
                        region_name="auto",
                        endpoint_url='https://storage.googleapis.com',  
                        aws_access_key_id=os.environ.get("HMAC_KEY"),
                        aws_secret_access_key=os.environ.get("HMAC_SECRET"))

    if not os.path.exists('material'):
        os.makedirs('material')

    for gcs_links in gcs_links_and_paths:
        for gcs_link in gcs_links:
            s3_key = gcs_link.replace("gs://", "")
            bucket, key = s3_key.split("/", maxsplit=1)

            # Extract filename from the key
            filename = key.split("/")[-1]

            # Specify full file path to save the file
            file_path = os.path.join('material', filename)

            s3.meta.client.download_file(
                Bucket=bucket,
                Key=key,
                Filename=file_path,
            )

gcs_links_and_paths = [['gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json','gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.flac']]
download_gcs(gcs_links_and_paths)


In [88]:
gcs_links_and_paths = [
    ("gs://example-bucket/example.json", "material/example.json"),
    ("gs://example-bucket/example.flac", "material/example.flac"),
]

for i in gcs_links_and_paths:
    print(i)

('gs://dubpro-ai-data/youtube_data/-_hv34i11Zg/-_hv34i11Zg.json', '/home/harsh/Documents/audio_trimming/material/_hv34i11Zg.flac')
('gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json', '/home/harsh/Documents/audio_trimming/material/--M8DKL-Vsw.flac')


In [90]:
# Getting the links ready to download flac and json files
gcs_links_and_paths = []
with open('file.txt') as fd:
    d = fd.read().splitlines()
d = ' '.join(d)

for e in d.split():
    fold_name = e.split('/')[-2]
    lst = []
    js = e + e.split('/')[-2] + '.json'
    fl = e + e.split('/')[-2] + '.flac'
    lst.append(js)
    lst.append(fl)
    gcs_links_and_paths.append(lst)
    break

gcs_links_and_paths

[['gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json',
  'gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.flac']]

In [81]:
gcs_links_and_paths

[['gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json',
  'gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.flac'],
 ['gs://dubpro-ai-data/youtube_data/-0eqyJdZozY/-0eqyJdZozY.json',
  'gs://dubpro-ai-data/youtube_data/-0eqyJdZozY/-0eqyJdZozY.flac'],
 ['gs://dubpro-ai-data/youtube_data/-1IpqxyE-tY/-1IpqxyE-tY.json',
  'gs://dubpro-ai-data/youtube_data/-1IpqxyE-tY/-1IpqxyE-tY.flac'],
 ['gs://dubpro-ai-data/youtube_data/-2-OnSsmvRI/-2-OnSsmvRI.json',
  'gs://dubpro-ai-data/youtube_data/-2-OnSsmvRI/-2-OnSsmvRI.flac'],
 ['gs://dubpro-ai-data/youtube_data/-2u3lHMoJK4/-2u3lHMoJK4.json',
  'gs://dubpro-ai-data/youtube_data/-2u3lHMoJK4/-2u3lHMoJK4.flac'],
 ['gs://dubpro-ai-data/youtube_data/-3Xepks2D54/-3Xepks2D54.json',
  'gs://dubpro-ai-data/youtube_data/-3Xepks2D54/-3Xepks2D54.flac'],
 ['gs://dubpro-ai-data/youtube_data/-3jqBi1a5bA/-3jqBi1a5bA.json',
  'gs://dubpro-ai-data/youtube_data/-3jqBi1a5bA/-3jqBi1a5bA.flac'],
 ['gs://dubpro-ai-data/youtube_data/-3vxz5PMShE/-3vxz5P

In [59]:
with open('file.txt') as fd:
    d = fd.read().splitlines()
d = ' '.join(d)

for e in d.split():
    js = e + e.split('/')[-2] + '.json'
    fl = e + e.split('/')[-2] + '.flac'
    break


gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.json 
 gs://dubpro-ai-data/youtube_data/--M8DKL-Vsw/--M8DKL-Vsw.flac


In [44]:
# Trim the audio
def trim_audio(input_flac_file, input_json_file):

    # Loading json file
    with open(input_json_file) as fd:
        info = json.load(fd)

    # Read the audio file
    data, samplerate = sf.read(input_flac_file)

    # Check if the audio is mono
    if len(data.shape) == 1:

        for idx, item in enumerate(info):
            
            start_time = item["start"]
            end_time = item["end"]
            
            # Calculate start and end samples
            start_sample = int(start_time * samplerate)
            end_sample = int(end_time * samplerate)

            # Trim the audio
            trimmed_data = data[start_sample:end_sample]

            # Write the trimmed audio to a new file

            if not os.path.exists('output_file'):
                os.makedirs('output_file')
            basename = os.path.basename('/home/harsh/Documents/audio_trimming/material/youtube_data_--M8DKL-Vsw_--M8DKL-Vsw.flac')
            path_to_otpt_audio = 'output_file/' + basename[:-5] + '_' + str(idx+1) + '.flac'
            
            sf.write(path_to_otpt_audio, trimmed_data, samplerate)

# input_flac_file = '/home/harsh/Documents/audio_trimming/material/_hv34i11Zg.flac'
# input_json_file = '/home/harsh/Documents/audio_trimming/material/_hv34i11Zg.json'
trim_audio(input_flac_file, input_json_file)